In [1]:
import import_ipynb
from maze_common import *
import time
import random as rnd
import numpy as np
import copy

importing Jupyter notebook from maze_common.ipynb


In [8]:
maze = Maze(40, .30)
solvable = maze.isSolvable()
print("Is solvable?", solvable)
print("Obstacles?", maze.obstacles.shape[0])

Is solvable? True
Obstacles? 451


In [9]:
if (solvable):
    startTime = time.time()
    shortestPath, expandedCells = shortestPathSearch(maze)
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")

Expanded cells: 1132
Shortest path length: 81 
Shortest path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 9), (6, 9), (6, 10), (6, 11), (6, 12), (5, 12), (5, 13), (5, 14), (6, 14), (6, 15), (6, 16), (6, 17), (6, 18), (6, 19), (7, 19), (8, 19), (8, 20), (8, 21), (9, 21), (9, 22), (9, 23), (10, 23), (11, 23), (12, 23), (13, 23), (14, 23), (15, 23), (16, 23), (17, 23), (17, 24), (17, 25), (18, 25), (18, 26), (18, 27), (19, 27), (20, 27), (20, 28), (21, 28), (21, 29), (22, 29), (23, 29), (24, 29), (24, 30), (25, 30), (26, 30), (26, 31), (26, 32), (27, 32), (28, 32), (29, 32), (29, 33), (30, 33), (31, 33), (32, 33), (32, 34), (33, 34), (34, 34), (35, 34), (35, 35), (36, 35), (36, 36), (36, 37), (36, 38), (36, 39), (37, 39), (38, 39), (39, 39)]


In [10]:
def manhattanDistance(coords, maze, visited):
    cellRow, cellCol = coords
    return ((maze.dim - 1) - cellRow) + ((maze.dim - 1) - cellCol), 0

if (solvable):
    startTime = time.time()
    shortestPath, expandedCells = shortestPathSearch(maze, heuristicFunction = manhattanDistance)
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")

Expanded cells: 618
Shortest path length: 81 
Shortest path: [(0, 0), (0, 1), (1, 1), (2, 1), (3, 1), (3, 2), (4, 2), (5, 2), (5, 3), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (7, 9), (7, 10), (7, 11), (8, 11), (8, 12), (8, 13), (7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (8, 19), (9, 19), (9, 20), (9, 21), (9, 22), (9, 23), (10, 23), (11, 23), (12, 23), (13, 23), (14, 23), (15, 23), (15, 24), (15, 25), (16, 25), (17, 25), (18, 25), (19, 25), (20, 25), (21, 25), (21, 26), (22, 26), (22, 27), (22, 28), (22, 29), (23, 29), (24, 29), (24, 30), (25, 30), (26, 30), (26, 31), (26, 32), (27, 32), (28, 32), (28, 33), (29, 33), (29, 34), (30, 34), (31, 34), (32, 34), (33, 34), (34, 34), (35, 34), (35, 35), (35, 36), (36, 36), (36, 37), (36, 38), (37, 38), (37, 39), (38, 39), (39, 39)]


In [11]:
def thinMaze(maze, fractionRemove):
    thinMaze = Maze(1, 0)
    thinMaze.board = copy.deepcopy(maze.board)
    thinMaze.obstacles = copy.deepcopy(maze.obstacles)
    thinMaze.dim = maze.dim
    numRemove = round(fractionRemove * thinMaze.obstacles.shape[0])
    for i in range(0,numRemove):
        indexRemove = int(rnd.random() * thinMaze.obstacles.shape[0])
        obstacleX, obstacleY = thinMaze.obstacles[indexRemove]
        thinMaze.obstacles = np.delete(thinMaze.obstacles,indexRemove, axis=0)
        thinMaze.board[obstacleX, obstacleY] = Cell.OPEN
    return thinMaze
    
thinnedMaze = thinMaze(maze, .5)
print("Original obstacles: ", maze.obstacles.shape[0])
print("Thinned obstacles: ", thinnedMaze.obstacles.shape[0])
# print("Thinned maze:\n", thinnedMaze.board)

Original obstacles:  451
Thinned obstacles:  225


In [12]:
def thinnedMazeShortestPathLength(coords, maze, visited):
    cellRow, cellCol = coords
    shortestPathLength = 0
    expandedCells = 0
    if ((cellRow, cellCol)) not in visited:
        shortestPath, expandedCellsBySearch = shortestPathSearch(thinnedMaze, startCoords = (cellRow, cellCol))
        shortestPathLength = len(shortestPath)
        expandedCells = expandedCellsBySearch
        for i in range(0, shortestPathLength):
            row, col = shortestPath[i]
            if ((row, col)) in visited: 
                break
            else:
                visited[(row, col)] = shortestPathLength - i - 1
    else:
        shortestPathLength = visited[(cellRow, cellCol)]
    return shortestPathLength, expandedCells

if (solvable):
    startTime = time.time()
    shortestPath, expandedCells = shortestPathSearch(maze, heuristicFunction = thinnedMazeShortestPathLength)
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")

Expanded cells: 175204
Shortest path length: 81 
Shortest path: [(0, 0), (1, 0), (2, 0), (2, 1), (3, 1), (3, 2), (3, 3), (4, 3), (4, 4), (5, 4), (6, 4), (6, 5), (6, 6), (7, 6), (7, 7), (7, 8), (7, 9), (7, 10), (8, 10), (8, 11), (8, 12), (8, 13), (7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (8, 19), (8, 20), (9, 20), (9, 21), (9, 22), (10, 22), (11, 22), (12, 22), (13, 22), (13, 23), (14, 23), (15, 23), (16, 23), (17, 23), (17, 24), (18, 24), (19, 24), (19, 25), (20, 25), (21, 25), (21, 26), (22, 26), (22, 27), (22, 28), (23, 28), (23, 29), (24, 29), (25, 29), (25, 30), (26, 30), (26, 31), (26, 32), (27, 32), (28, 32), (29, 32), (30, 32), (31, 32), (31, 33), (31, 34), (32, 34), (33, 34), (34, 34), (35, 34), (35, 35), (36, 35), (36, 36), (36, 37), (36, 38), (37, 38), (37, 39), (38, 39), (39, 39)]


In [13]:
def findNeighboringOpenCoordsIncludingDiagonals(coords, maze):
    cellRow, cellCol = coords
    neighbors = findNeighboringOpenCoords(coords, maze)
    potentialDiagonalNeighbors = [(cellRow - 1, cellCol - 1), (cellRow - 1, cellCol + 1), (cellRow + 1, cellCol - 1), (cellRow + 1, cellCol + 1)]
    for potentialDiagonalNeighbor in potentialDiagonalNeighbors:
        row, col = potentialDiagonalNeighbor
        if (row >= maze.dim or row < 0 or col >= maze.dim or col < 0 or maze.board[row,col] != Cell.OPEN):
            continue
        neighbors.append((row, col))
    return neighbors

def diagonalTravelShortestPathLength(coords, maze, visited):
    cellRow, cellCol = coords 
    shortestPathLength = 0
    expandedCells = 0
    if ((cellRow, cellCol)) not in visited:
        shortestPath, expandedCellsBySearch = shortestPathSearch(maze, startCoords = (cellRow, cellCol), findNeighborsFunction = findNeighboringOpenCoordsIncludingDiagonals)
        shortestPathLength = len(shortestPath)
        expandedCells = expandedCellsBySearch
        for i in range(0, shortestPathLength):
            row, col = shortestPath[i]
            if ((row, col)) in visited: 
                break
            else:
                visited[(row, col)] = shortestPathLength - i - 1
    else:
        shortestPathLength = visited[(cellRow, cellCol)]
    return shortestPathLength, expandedCells

if (solvable):
    startTime = time.time()
    shortestPath, expandedCells = shortestPathSearch(maze, heuristicFunction = diagonalTravelShortestPathLength)
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    boardWithPath = copy.deepcopy(maze.board)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")

Expanded cells: 388265
Shortest path length: 81 
Shortest path: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (3, 3), (3, 4), (4, 4), (5, 4), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (6, 12), (5, 12), (5, 13), (5, 14), (6, 14), (6, 15), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (8, 19), (8, 20), (8, 21), (9, 21), (9, 22), (10, 22), (11, 22), (12, 22), (13, 22), (13, 23), (14, 23), (15, 23), (16, 23), (17, 23), (17, 24), (18, 24), (18, 25), (18, 26), (18, 27), (19, 27), (20, 27), (20, 28), (21, 28), (22, 28), (23, 28), (23, 29), (24, 29), (24, 30), (25, 30), (25, 31), (25, 32), (26, 32), (27, 32), (28, 32), (29, 32), (30, 32), (31, 32), (31, 33), (32, 33), (33, 33), (33, 34), (34, 34), (35, 34), (35, 35), (35, 36), (36, 36), (36, 37), (36, 38), (37, 38), (37, 39), (38, 39), (39, 39)]
